### Fact Check Assistant: Setup Strands Agent


## 1. Setup and Installation

First, let's install the required libraries:


In [ ]:
%pip install --upgrade -q -r ../src/frontend/requirements.txt

## 2. Import Libraries and Configure Logging


In [3]:
import time
import logging
import sys
from strands.tools.mcp import MCPClient
from mcp import stdio_client, StdioServerParameters

from strands import Agent, tool
from strands_tools import calculator

In [4]:
# Set up logging specifically for Strands components
loggers = [
    "strands",
    "strands.agent",
    "strands.tools",
    "strands.models",
    "strands.bedrock",
]
for logger_name in loggers:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.INFO)
    # Add console handler if not already present
    if not logger.handlers:
        handler = logging.StreamHandler()
        formatter = logging.Formatter(
            "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
        )
        handler.setFormatter(formatter)
        logger.addHandler(handler)

## 3. Create Agent for Mathematical Calculations

We will be creating an agent to verify numerical claims and calculations, equipped with a retrieve tool to access the previously established Knowledge Base. This agent will analyze mathematical assertions, cross-reference them with reliable sources, and provide accurate fact-checking for numerical data and computational results.


In [5]:
MATH_ASSISTANT_SYSTEM_PROMPT = """
You are math wizard, a specialized mathematics applying mathematics across a wide variety of domains.

People need your opinion on the validity of statements they pass to you. Use the tools at your disposal to logically break down a statement to determine whether the statement is true or false. 

Check the statement for mathematical and process errors.

Your capabilities include:
1. Mathematical Operations:
   - Arithmetic calculations
   - Algebraic problem-solving
   - Geometric analysis
   - Statistical computations

2. Teaching Tools:
   - Step-by-step problem solving
   - Visual explanation creation
   - Formula application guidance
   - Concept breakdown

3. Educational Approach:
   - Show detailed work
   - Explain mathematical reasoning
   - Provide alternative solutions
   - Link concepts to real-world applications

4. Australian Taxation Rules
   - Detailed knowledge of Australian Taxation Office Rules (https://www.ato.gov.au/)
   - Ability to perform income tax calculations
   - Ability to perform capital gains tax calculations
   - Analyse structure of personal finances to minimise tax

5. Mortgage Calculations
   - Detailed understanding of how mortgages work in Australia

Focus on clarity and systematic problem-solving while ensuring people understand the underlying concepts.
"""

In [6]:
@tool
def math_assistant(query: str) -> str:
    """
    Process and respond to math-related queries using a specialized math agent.

    Args:
        query: A mathematical question or problem from the user

    Returns:
        A detailed mathematical answer with explanations and steps
    """
    # Format the query for the math agent with clear instructions
    formatted_query = f"Please solve the following mathematical problem, showing all steps and explaining concepts clearly: {query}"

    try:
        # Create the math agent with calculator capability
        math_agent = Agent(
            model="anthropic.claude-3-5-haiku-20241022-v1:0",
            tools=[calculator],
            system_prompt=MATH_ASSISTANT_SYSTEM_PROMPT,
        )

        agent_response = math_agent(formatted_query)
        text_response = str(agent_response)

        if len(text_response) > 0:
            return text_response

        return "I apologize, but I couldn't solve this mathematical problem. Please check if your query is clearly stated or try rephrasing it."
    except Exception as e:
        # Return specific error message for math processing
        return f"Error processing your mathematical query: {str(e)}"

Test the Math Verification agent when validating numerical calculations and mathematical claims.


In [ ]:
response = math_assistant("What is 2 + 2?")

## 4. Create Agent for Researching Erroneous Claims

We will be creating an agent specialized in identifying and investigating false or misleading claims, integrated with the DuckDuckGo MCP server for real-time web search capabilities. This agent will leverage both the previously established Knowledge Base and DuckDuckGo's search functionality to research questionable assertions, identify factual inaccuracies, and provide evidence-based corrections. By combining local knowledge with dynamic web search results, the agent will systematically analyze claims, cross-reference them with multiple authoritative sources, and deliver comprehensive fact-checking results that explain why certain claims are erroneous and what the accurate information is.


In [8]:
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx",
            args=["duckduckgo-mcp-server"],
        )
    )
)

In [9]:
RESEARCH_ASSISTANT_SYSTEM_PROMPT = """
You are a specialized fact-checking agent. Your role is to:

1. Analyze Claims:
   - Evaluate the truthfulness of statements
   - Identify verifiable facts vs opinions
   - Assess credibility of information

2. Provide Evidence:
   - Search for reliable sources
   - Cross-reference multiple sources
   - Cite authoritative references

3. Deliver Verdict:
   - Clear TRUE/FALSE/PARTIALLY TRUE assessment
   - Confidence level in your assessment
   - Additional context and nuances

4. Explain Reasoning:
   - Break down your fact-checking process
   - Highlight key evidence
   - Note any limitations or uncertainties

Always provide balanced, evidence-based analysis with proper context.
"""


In [10]:
@tool
def research_assistant(query: str) -> str:
    """
    Fact-check a statement and return whether it's true or false with supporting context.

    Args:
        statement: The statement or claim to fact-check

    Returns:
        A fact-check assessment with verdict (TRUE/FALSE/PARTIALLY TRUE) and supporting evidence
    """
    formatted_query = f"Please fact-check this statement and provide a clear verdict (TRUE/FALSE/PARTIALLY TRUE) with supporting evidence and context: {query}"

    try:
        print("Routed to Fact Checker")
        with stdio_mcp_client:
            # Get the tools from the MCP server
            tools = stdio_mcp_client.list_tools_sync()

            # Create an agent with MCP tools and fact-checking prompt
            fact_checker_agent = Agent(
                model="anthropic.claude-3-5-haiku-20241022-v1:0",
                tools=tools,
                system_prompt=RESEARCH_ASSISTANT_SYSTEM_PROMPT,
            )
            agent_response = fact_checker_agent(formatted_query)
            text_response = str(agent_response)

            if len(text_response) > 0:
                return text_response

            return "I apologize, but I couldn't fact-check this statement. Please provide a clear, specific claim to verify."
    except Exception as e:
        return f"Error during fact-checking: {str(e)}"

In [ ]:
response = research_assistant("The Great Wall of China is visible from the Moon.")

## 5. Create Agent for Detecting Erroneous Claims

We will be creating an agent specialized in researching erroneous or misleading claims in text. The agent will flag instances of fabrication, manipulation, or critical omissions based on its existing knowledge.


In [11]:
CLAIM_DETECTION_ASSISTANT_SYSTEM_PROMPT = """
You are an assistant specialized in identifying erroneous or misleading claims in text.

Instructions:
- Break down the input text into separate statements.
- For each statement, determine if it contains misinformation, disinformation, or is otherwise clearly false or misleading.
- Fact-check statements using general knowledge and logical reasoning. Look for fabrication, manipulation, or critical omissions.

Output:
- Return a numbered list of all statements you identify as erroneous, incorrect, or misleading.
- Each listed item should be the erroneous claim as a direct quotation.
- If there are no erroneous statements, return 'No erroneous claims found.'
- Do not return any explanations, extra text, or formatting.

Example 1
Input:
Climate change is a hoax invented by scientists. The Great Wall of China is visible from space.

Output:
1. "Climate change is a hoax invented by scientists."
2. "The Great Wall of China is visible from space."

Example 2
Input:
The Pacific Ocean is the largest ocean on Earth. Drinking bleach can cure illnesses.

Output:
1. "Drinking bleach can cure illnesses."

Example 3
Input:

Output:
No erroneous claims found.
"""

In [12]:
@tool
def claim_detection_assistant(query: str) -> str:
    """
    Analyze free text, segment it into statements, and flag each for erroneous claim.

    Args:
        query: Free-form text containing one or more statements

    Returns:
        The potentially erroneous claims.
    """
    formatted_query = f"Please identify erroneous or misleading claims with supporting evidence and context: {query}"

    try:
        research_agent = Agent(
            model="anthropic.claude-3-5-haiku-20241022-v1:0",
            system_prompt=CLAIM_DETECTION_ASSISTANT_SYSTEM_PROMPT,
        )

        agent_response = research_agent(formatted_query)
        text_response = str(agent_response)

        if len(text_response) > 0:
            return text_response

        return "I apologize, but I couldn't identify any erroneous claims. Please check if your query is clearly stated or try rephrasing it."
    except Exception as e:
        # Return specific error message for math processing
        return f"Error identifying erroneous claims: {str(e)}"

In [ ]:
response = claim_detection_assistant("""
In a historic milestone for both science and national pride, experts reported today that astronauts aboard lunar missions have been able to identify human-made structures on Earth with the naked eye. Among these, The Great Wall of China is visible from the Moon. 
This remarkable achievement underscores the sheer scale and enduring legacy of the iconic wall, which winds its way more than 20,000km across the vast landscapes of northern China. Lunar explorers described how, despite Earth's cloud cover and atmospheric distortion, the serpentine line of the Great Wall stands out against the surrounding terrain when observed from their vantage point on the lunar surface.

International observers hailed this revelation as a testament to both ancient engineering and the advancements in observational capabilities made possible by modern space exploration technologies. According to the report, such visibility is not only a source of pride for the people of China but also an enduring symbol of the achievements of human civilization.
""")

## 6. Create Supervisor Agent for Fact Checking

We will be creating a supervisor agent that orchestrates multiple specialized sub-agents to efficiently detect and remediate erroneous claims. This agent will serve as the central coordinator, methodically analyzing incoming text, determining which specialized agent is best suited for each verification task, and routing queries accordingly.

The supervisor will maintain a structured workflow, collecting outputs from each sub-agent and synthesizing them into a comprehensive assessment. Without performing direct fact-checking itself, this agent will ensure that all claims are properly evaluated by the appropriate specialized agents, resulting in a thorough and systematic verification process that identifies all instances of misinformation or misleading content.


In [13]:
SUPERVISOR_SYSTEM_PROMPT = """
You are a sophisticated misinformation detection orchestrator that coordinates specialized agents for comprehensive content analysis and fact-checking. Your primary mission is to detect, analyze, and verify information for potential misinformation.

## Available Specialized Agents

You have access to these specialized agents:

1. **claim_detection_assistant** - Claim Detection Assistant
   - Breaks down input text into separate statements
   - Identifies potentially false or misleading claims
   - Returns a numbered list of erroneous claims as direct quotations
   - Use for: Initial content analysis, statement extraction, identifying suspicious claims

2. **research_assistant** - External Fact Checker Assistant
   - Performs deep fact-checking with evidence using DuckDuckGo search
   - Searches reliable sources and cross-references information
   - Provides TRUE/FALSE/PARTIALLY TRUE verdicts with supporting evidence
   - Use for: Verifying specific claims, finding authoritative sources, detailed fact-checking

3. **math_assistant** - Math Wizard
   - Validates mathematical claims and calculations
   - Checks numerical accuracy and statistical validity
   - Analyzes quantitative statements for errors
   - Use for: Mathematical fact-checking, statistical claims, numerical verification

## Orchestration Strategy

For comprehensive misinformation detection, follow this multi-step approach:

### Step 1: Initial Analysis
- Use **claim_detection_assistant** to segment content and identify potentially problematic statements
- This provides a structured list of all claims that need verification

### Step 2: Specialized Verification
- For mathematical/statistical claims → Use **math_assistant**
- For factual claims requiring evidence → Use **research_assistant**
- For complex claims → Coordinate multiple agents as needed

### Step 3: Synthesis
- Combine results from all agents
- Provide concise assessment with evidence
- Highlight the most concerning misinformation risks

## Trigger Keywords for Misinformation Analysis

Always initiate the multi-agent workflow when you detect:
- "fact-check", "verify claims", "check if true"
- "misinformation", "disinformation", "fake news", "suspicious content"
- Any request to evaluate content truthfulness

## Best Practices

1. **Always start with claim_detection_assistant** for content segmentation
2. **Use research_assistant** for claims requiring external verification
3. **Use math_assistant** for any numerical or statistical claims
4. **Coordinate multiple agents** for comprehensive analysis
5. **Provide clear, evidence-based conclusions**

Your goal is to provide concise, accurate misinformation detection by leveraging the specialized capabilities of each agent in a coordinated manner.
"""


In [14]:
## Create Supervisor Agent for Fact Checking
@tool
def fact_check_supervisor(query: str) -> str:
    """
    Process queries through the multi-agent fact-checking system.

    Args:
        query: The text to analyze for factual accuracy

    Returns:
        str: A comprehensive fact-check assessment
    """
    # Use MCP client context manager to ensure session is active
    with stdio_mcp_client:
        try:
            # Combine custom tools with MCP tools
            all_tools = [
                math_assistant,
                claim_detection_assistant,
                research_assistant,
            ]

            # Create the supervisor agent within MCP context
            supervisor = Agent(
                model="anthropic.claude-3-5-haiku-20241022-v1:0",
                system_prompt=SUPERVISOR_SYSTEM_PROMPT,
                tools=all_tools,
            )

            # Process the query within the same MCP context
            return supervisor(
                f"Provide a comprehensive fact-check assessment for this text: {query}"
            )

        except Exception as e:
            logging.error(f"Error processing query with MCP tools: {str(e)}")

            # Fallback to supervisor without MCP tools
            supervisor = Agent(
                model="anthropic.claude-3-5-haiku-20241022-v1:0",
                system_prompt=SUPERVISOR_SYSTEM_PROMPT,
                tools=[
                    math_assistant,
                    claim_detection_assistant,
                ],
            )

            return supervisor(
                f"Provide a comprehensive fact-check assessment for this text: {query}"
            )

## 7. Test the Complete Multi-Agent System

Now let's test the complete multi-agent system with various queries:


In [ ]:
# Test the supervisor agent
print("\n\nInvoking fact-checking supervisor agent...\n\n")

requests = [
    "The Great Wall of China is visible from the Moon with the naked eye.",
    "The average mortgage interest rate in Australia is 1.5%, which means on a $500,000 loan over 30 years, you'll pay just $250,000 in total interest.",
    "Drinking lemon water every morning can cure diabetes by neutralizing blood sugar levels.",
    "The Earth is flat and this has been proven by multiple scientific studies.",
    "If you invest $10,000 at a 7% annual return compounded monthly, you'll have approximately $20,000 after 10 years.",
]

results = []

for request in requests:
    print(f"\n\nRequest: {request}\n\n")
    results.append(fact_check_supervisor(request))
    time.sleep(10)  # Allow time between requests

In [ ]:
for result in results:
    print(result, "\n\n", "*" * 50, "\n\n")

## 8. Interactive Chat Loop

Let's create an interactive chat loop to test the system with custom queries:


In [ ]:
import io
import sys

print("Fact Check Assistant Chat - Type 'quit' to exit")
print("=" * 50)

while True:
    user_input = input("\nYou: ").strip()

    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break

    if not user_input:
        continue

    try:
        # Capture stdout to prevent double printing
        old_stdout = sys.stdout
        sys.stdout = io.StringIO()

        response = fact_check_supervisor(user_input)

        # Restore stdout
        captured_output = sys.stdout.getvalue()
        sys.stdout = old_stdout

        print(f"\nAssistant: {response}")
    except Exception as e:
        sys.stdout = old_stdout
        print(f"\nError: {str(e)}")
        print("Please try again or type 'quit' to exit.")